In [1]:
import pandas as pd
import numpy as np

In [3]:
# CSV 파일 경로
file_path = 'ts2000_y_processed_v1.csv'
df = pd.read_csv(file_path)
df.head()

,Unnamed: 0,corp_nm,stock_code,year,자산총계_당기,자산총계_전기,비유동자산_당기,비유동자산_전기,유동자산_당기,유동자산_전기,...,당기순이익_전기,재고자산_당기,재고자산_전기,매출채권_당기,매출채권_전기,매출채권및기타유동채권_당기,매출채권및기타유동채권_전기,총현금흐름_당기,총현금흐름_전기,is_defaulted
0,0,(주)CMG제약,58820,2015,56591972.0,58127098.0,30261207.0,29767275.0,26330765.0,28359823.0,...,171502.0,5455617.0,4913103.0,15838056.0,16787073.0,12405117.0,13533700.0,-6259636.0,-12880662.0,0
1,1,(주)CMG제약,58820,2016,110979867.0,56591972.0,32207666.0,30261207.0,78772201.0,26330765.0,...,-3648668.0,6393756.0,5455617.0,14435808.0,15838056.0,12748287.0,12405117.0,-413245.0,-6259636.0,0
2,2,(주)CMG제약,58820,2017,109568153.0,110979867.0,28340823.0,32207666.0,81227330.0,78772201.0,...,1693463.0,7904518.0,6393756.0,16768648.0,14435808.0,15507312.0,12748287.0,1652143.0,-413245.0,0
3,3,(주)CMG제약,58820,2018,114379880.0,109568153.0,33009465.0,28340823.0,81370415.0,81227330.0,...,2834901.0,8572582.0,7904518.0,19030822.0,16768648.0,17706490.0,15507312.0,9665998.0,1652143.0,0
4,4,(주)CMG제약,58820,2019,192243166.0,114379880.0,41659100.0,33009465.0,150584066.0,81370415.0,...,6065026.0,9048933.0,8572582.0,21676515.0,19030822.0,21160960.0,17706490.0,5105725.0,9665998.0,0


In [4]:
# 열별 결측치 개수 확인
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]
missing_values.sort_values(ascending=False)


투자활동현금흐름_전기       2608
유동부채_전기           2608
매출채권및기타유동채권_전기    2608
매출채권_전기           2608
재고자산_전기           2608
당기순이익_전기          2608
매출원가_전기           2608
영업이익_전기           2608
판매관리비_전기          2608
매출총이익_전기          2608
매출액_전기            2608
재무활동현금흐름_전기       2608
자산총계_전기           2608
영업활동현금흐름_전기       2608
현금및현금성자산_전기       2608
총현금흐름_전기          2608
유동자산_전기           2608
비유동부채_전기          2608
자본금_전기            2608
부채총계_전기           2608
비유동자산_전기          2608
자본총계_전기           2608
이익잉여금_전기          2608
유동부채_당기           1809
자본총계_당기           1809
당기순이익_당기          1809
유동자산_당기           1809
재고자산_당기           1809
매출채권_당기           1809
매출채권및기타유동채권_당기    1809
비유동자산_당기          1809
총현금흐름_당기          1809
매출원가_당기           1809
자본금_당기            1809
영업이익_당기           1809
판매관리비_당기          1809
매출총이익_당기          1809
이익잉여금_당기          1809
매출액_당기            1809
재무활동현금흐름_당기       1809
부채총계_당기           1809
투자활동현금흐름_당기       1809
영업활동현금흐름_당기       1809
비유동부채_당기   

In [6]:
# 기업별 그룹화
grouped = df.groupby('corp_nm')

processed_rows = []

for corp, group in grouped:
    group = group.copy()
    
    for col in group.columns:
        if col in ['corp_nm', 'year']:
            continue
        
        if group[col].isnull().any():
            total_count = len(group)
            valid_count = group[col].notnull().sum()
            valid_ratio = valid_count / total_count

            if valid_ratio >= 0.8:
                median_value = group[col].median()
                group[col].fillna(median_value, inplace=True)
            else:
                group = group[~group[col].isnull()]
    
    processed_rows.append(group)

# 결합하여 최종 정제된 데이터프레임 생성
df_cleaned = pd.concat(processed_rows, ignore_index=True)


In [9]:
# 수치형 열만 선택
numeric_cols = df_cleaned.select_dtypes(include=['float64', 'int64']).columns

# 복사본 생성
df_winsorized = df_cleaned.copy()

# 윈저라이징 비율 설정 (상하단 0.5%씩)
lower_percentile = 0.005
upper_percentile = 0.995

for col in numeric_cols:
    lower = df_winsorized[col].quantile(lower_percentile)
    upper = df_winsorized[col].quantile(upper_percentile)
    
    df_winsorized[col] = df_winsorized[col].clip(lower=lower, upper=upper)


In [11]:
# CSV로 저장
df_winsorized.to_csv('df_winsorized_output.csv', index=False)
